# Question :

 Assume there is a process with certain variability in the result. The operary
gives a recipe (R) for the process and then measures the result (M) and saves
everything
in a database of name "database_name" within a table of name "table_name".
example:


        table_name
        | id | R | M |
        | 01 | 1 | 0.90 |
        | 02 | 1 | 0.95 |
        | 03 | 2 | 2.00 |
        | 04 | 3 | 3.01 |
        | 05 | 3 | 3.03 |
        | 06 | 4 | 3.99 |
        | 07 | 4 | 3.90 |
        | 08 | 4 | 4.12 |
        | 09 | 4 | 4.20 |
        | 10 | 5 | 5.02 |



make a SQL statement to get the last n diferent recipes, showing in each case the
last entry if there is multiple entries with the same recipe.

Example 1:

input:
sql(1)

output:


        | 10 | 5 | 5.02 |


Example 2:

input:
sql(3)

output:

        | 05 | 3 | 3.03 |
        | 09 | 4 | 4.20 |
        | 10 | 5 | 5.02 |


consider:
- what if R and M exist in different tables with a connection table ( | conn_id | R_id
| M_id | )?

## Possible Solution with SSMS (Sql Server Management Studio)
* For this procedure, you need SQL and Sql Server Management Studio installed in your local machine

~~~sql


-- create first table as new
CREATE TABLE new (
	id int NOT NULL IDENTITY PRIMARY KEY,
	R int,
	M int
);

INSERT INTO new (R,M)
VALUES (2,10), (2,11), (3,12), (6,9), (6,40), (4,21), (4,23);



-- create a cte view as select the rows as we need
WITH cte1 as (
SELECT id, R, M
FROM new
WHERE id IN (
    SELECT MAX(id)
    FROM new
    GROUP BY R))
	/*Here I am slicing the first 3 rows of unique Reciepes*/
SELECT TOP(3) id, R, M
FROM cte1 ORDER BY id DESC;



-- Create second table as new1
CREATE TABLE new1 (
	id int NOT NULL IDENTITY PRIMARY KEY,
	R int,
	M int
);

INSERT INTO new1 (R,M)
VALUES (2,10), (2,11), (3,12), (6,9), (6,40), (4,21), (4,23);


-- create third table from previous new & new1 as nst
-- Here, I am assuming that the 'id' column is common in the both tables new & new1
-- taking id, R from one table & id, M from another table, and creating a table based on commom id in the both tables
SELECT new1.id conn_id, new1.R,  new.M INTO nst FROM new1, new
WHERE new1.id = new.id;

-- create a cte2 view as select the rows as we need
with cte2 as (
SELECT id, R, M
FROM nst
WHERE id IN (
    SELECT MAX(id)
    FROM nst
    GROUP BY R))
	/*Here I am slicing the first 2 rows of unique Reciepes*/
SELECT TOP(1) id, R, M
FROM cte2 ORDER BY id DESC;

~~~






## Possible Solution with Sqlite3 & Python

In [ ]:
import sqlite3

# Creating a connection with temporary database for each executiobn
conn = sqlite3.connect(':memory:')
c = conn.cursor()

# Creating table called Recipe
c.execute("""
CREATE TABLE Recipe(
    R_id INTEGER PRIMARY KEY,
    R INTEGER,
    M INTEGER
)
""")

conn.commit()

# Populating the Recipe table
c.execute("""
INSERT INTO Recipe (R,M) VALUES (3,12),(3,11), (4,12), (4,20), (5,9),(5,21)
""")

conn.commit()

c.execute("""SELECT * 
FROM Recipe LIMIT 3""")

res = c.fetchall()
for i in res:
    print(i)
conn.commit()



# Define `sql()` function.
def sql(n_rows):

    """

    Statement could be defined with individual parts such as-

    cte = "WITH cte1 as (SELECT R_id, R, M FROM Recipe "
    cond = "WHERE R_id IN (SELECT MAX(R_id) FROM Recipe GROUP BY R))"
    select = f"SELECT * FROM cte1 ORDER BY R_id DESC LIMIT {n_rows}"
    command = (f'{cte}{cond}{select}')
    
    c.execute(command)
   
    If we want to use a python variable as input in sql statement, the whole 
    statement should in single line. So, I complie the whole sql statement here

    """
    statement = f"WITH cte1 as (SELECT R_id, R, M FROM Recipe\
        WHERE R_id IN (SELECT MAX(R_id) FROM Recipe GROUP BY R))\
        SELECT * FROM cte1 ORDER BY R_id DESC LIMIT {n_rows}"
    
    c.execute(statement)
    res = c.fetchall()
    print(" R_id,R, M\n")
    for i in res:
        print(i)


print("\nRecipe from the First Tbale : Recipe\
(made from two different tables")

# Calling the function with argument
sql(2)

#Create Second Table; Recipe
c.execute("""
CREATE TABLE Recipe2(
    R_id INTEGER PRIMARY KEY,
    R INTEGER,
    M INTEGER
)
""")

# Populate the second table; Recipe2
c.execute("INSERT INTO Recipe2 (R,M)\
    VALUES (2,10), (2,11), (3,12), (6,9), (6,40), (4,21), (4,23);")

# Checking the rows in the Recipe2 table
c.execute("SELECT * FROM Recipe2")
res = c.fetchall()
print("R_id,R,M\n")
for i in res:
    print(i)
conn.commit()

"""
    Create a Mixed Table from Recipe & Recipe2, so that we can extract data from
    both tables.
"""

# Complie the statement for mixed_table
statement = "CREATE TABLE mixed_table AS\
  SELECT Recipe2.R_id, Recipe2.R, Recipe.M\
  FROM Recipe2,Recipe\
  WHERE Recipe2.R_id = Recipe.R_id"

c.execute(statement)
c.execute("SELECT * FROM mixed_table LIMIT 3")
res = c.fetchall()

print(" First few rows of mixed_table that has been\
created from combination Recipe & Recipe2 tables")
for i in res:
    print(i)
conn.commit()

## Check the column name in the new created table
c.execute("SELECT c.name FROM pragma_table_info('mixed_table') c")
c.fetchall()

## Scuceesful attempt
def sql(n_rows):

    """

    Statement could be defined with individual parts such as-
    cte = "WITH cte1 as (SELECT R_id, R, M FROM Recipe "
    cond = "WHERE R_id IN (SELECT MAX(R_id) FROM Recipe GROUP BY R))"
    select = f"SELECT * FROM cte1 ORDER BY R_id DESC LIMIT {n_rows}"
    command = (f'{cte}{cond}{select}')
    
    c.execute(command)
   
    If we want to use a python variable as input in sql statement, the whole 
    statement should in single line. So, I complie the whole sql statement here

    """
    statement = f"WITH cte1 as (SELECT R_id, R, M FROM mixed_table\
        WHERE R_id IN (SELECT MAX(R_id) FROM mixed_table GROUP BY R))\
        SELECT * FROM cte1 ORDER BY R_id DESC LIMIT {n_rows}"
    
    c.execute(statement)
    res = c.fetchall()

    # print("R_id,R, M\n")
    # for i in res:
    #     print(i)


print("\nRecipe from the mixed_table\
(made from two different tables")

sql(3)
